In [3]:
# libraries
import pandas as pd
import numpy as np
import nltk
import os
import datetime as dt
from urllib.request import urlopen
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

In [4]:

from time import strptime


dat170 = pd.read_csv("r6-170-data-1-2.csv")

description = dat170['Event Description']


dat170.head()

,#,Summary Nr,Event Date,Report ID,Fat,SIC,Event Description,InvURL
0,1,144118.01,"Monday, February 28, 2022",626700,NaN,NaN,Employee Amputates Arm In Rotating Belt,https://www.osha.gov/pls/imis/accidentsearch.a...
1,2,144150.01,"Monday, February 28, 2022",627700,X,NaN,Employee Is Killed When Struck By Casing,https://www.osha.gov/pls/imis/accidentsearch.a...
2,3,144266.01,"Monday, February 28, 2022",626600,NaN,NaN,Employee Faints While Unloading Trailer,https://www.osha.gov/pls/imis/accidentsearch.a...
3,4,144235.01,"Saturday, February 26, 2022",627100,NaN,NaN,Employee'S Finger Is Amputated When Hand Sucke...,https://www.osha.gov/pls/imis/accidentsearch.a...
4,5,144787.01,"Friday, February 25, 2022",625700,NaN,NaN,Employee Falls And Injures Head,https://www.osha.gov/pls/imis/accidentsearch.a...


In [5]:
# we need to parse the date into 2 columnns, day and date.


from tkinter import E


daydate = dat170['Event Date']
daydate = daydate.to_string()
daydate = daydate.split('\n')
newdate = [i.split(', ', 1)[1] for i in daydate]
newdate 
#eventdate = str()
eventmonthday = [i.split(',',1)[0] for i in newdate]
eventyear = [i.split(', ',1)[1] for i in newdate]
eventmonth = [i.split(' ',1)[0] for i in eventmonthday]
eventday = [i.split(' ',1)[1] for i in eventmonthday]
eventday = [str(i).zfill(2) for i in eventday]
eventday
datedf =pd.DataFrame([eventday, eventmonth, eventyear])
datearray = np.array(datedf.T)
datearray
finaldate = [dt.datetime.strptime(str(i), "['%d' '%B' '%Y']") for i in datearray]



In [6]:

data = pd.DataFrame(dat170.loc[:, dat170.columns != 'Event Date'])
#data = pd.DataFrame(data.loc[:, data.columns != 'InvURL'])
#data = pd.DataFrame(data.loc[:, data.columns != 'SIC'])
data['Date'] = finaldate
data['Fat'] = data['Fat'].fillna(0)
data['Fat'] = data['Fat'].replace('X', 1)
data['InvURL'] = data['InvURL'].astype(str) + '5'

data.head()




,#,Summary Nr,Report ID,Fat,SIC,Event Description,InvURL,Date
0,1,144118.01,626700,0,NaN,Employee Amputates Arm In Rotating Belt,https://www.osha.gov/pls/imis/accidentsearch.a...,2022-02-28
1,2,144150.01,627700,1,NaN,Employee Is Killed When Struck By Casing,https://www.osha.gov/pls/imis/accidentsearch.a...,2022-02-28
2,3,144266.01,626600,0,NaN,Employee Faints While Unloading Trailer,https://www.osha.gov/pls/imis/accidentsearch.a...,2022-02-28
3,4,144235.01,627100,0,NaN,Employee'S Finger Is Amputated When Hand Sucke...,https://www.osha.gov/pls/imis/accidentsearch.a...,2022-02-26
4,5,144787.01,625700,0,NaN,Employee Falls And Injures Head,https://www.osha.gov/pls/imis/accidentsearch.a...,2022-02-25


In [7]:
link = str(data['InvURL'][1])
response = urlopen(link)
content = response.read()
#print(content)
parsed = BeautifulSoup(content)
parsed_table = parsed.body.find('div', attrs={'class':'table-responsive'})
#parsed_table = parsed_table.find_all('td')
parsed_table


<div class="table-responsive"><table class="tablei_100 table-borderedi_100 table-striped">
<tr><td colspan="8"><div class="text-center"><strong>Accident: 144150.015 -- Report ID: 0627700 -- Event Date: 02/28/2022</strong></div></td></tr>
<tr><th colspan="1">Inspection</th><th colspan="1" nowrap="">Open Date</th><th colspan="1">SIC</th><th colspan="5">Establishment Name</th></tr>
<tr><td colspan="1"><a href="establishment.inspection_detail?id=1580876.015" title="1580876.015">1580876.015</a></td><td colspan="1">03/01/2022</td><td colspan="1"><a href="sic_manual.display?id=&amp;tab=description" title=""></a></td><td colspan="5">Franks International Llc</td></tr>
<tr><td colspan="8">
At 4:45 p.m. on February 28, 2022, an employee was performing intermediate casin g work at an oil/gas drilling wellsite. The 44 foot joint of casing fell from th e elevators, striking the employee.  The employee suffered multiple fractures an d was killed.
</td></tr>
<tr><td colspan="8"><div class="text-center

In [8]:
insplink = 'https://www.osha.gov/pls/imis/establishment.inspection_detail?id=' + parsed_table.a.text
insplink
#link
inspresponse = urlopen(insplink)
inspcontent = inspresponse.read()
parsed_insp = BeautifulSoup(inspcontent)
parsed_insp_details = parsed_insp.body.find('div', attrs={'class':'container', 'id':'maincontain'})
parsed_insp_table = parsed_insp_details.find('table', attrs={'class':'table-borderedu'})
naics = parsed_insp_details.find('td', attrs={'nowrap':"", 'colspan':"4"})
naics



<td colspan="4">NAICS: 213112/Support Activities for Oil and Gas Operations</td>

In [10]:


# insplinks = list()

# for x in data['#']:
    # try:
    #     link = str(data['InvURL'][x])
    #     response = urlopen(link)
    #     content = response.read()
    #     parsed = BeautifulSoup(content)
    #     parsed_table = parsed.body.find('div', attrs={'class':'table-responsive'})
    #     insplink = 'https://www.osha.gov/pls/imis/establishment.inspection_detail?id=' + parsed_table.a.text
    #     insplinks.append(insplink)
    # except:
    #     insplinks.append("NA")
    #     pass





In [12]:
inspnodf = pd.DataFrame(insplinks)
inspnodf.to_csv('INSPLINKS.csv')

In [ ]:
    # inspresponse = urlopen(insplink)
    # inspcontent = inspresponse.read()
    # parsed_insp = BeautifulSoup(inspcontent)
    # parsed_insp_details = parsed_insp.body.find('div', attrs={'class':'container', 'id':'maincontain'})
    # #parsed_insp_table = parsed_insp_details.find('table', attrs={'class':'table-borderedu'})
    # naics = parsed_insp_details.find('td', attrs={'nowrap':"", 'colspan':"4"})
    # naicslist.append(str(naics))
